In [ ]:
!pip install findspark

In [ ]:
mount_point = "/content/drive"
from google.colab import drive
drive.mount(mount_point)

Mounted at /content/drive


In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_unixtime
from datetime import datetime
from pyspark.sql.window import Window
from pyspark.sql.functions import col, date_format, min, max, count, sum, avg, count_distinct, month, to_date, dense_rank, row_number, regexp_extract, explode, split, first, from_unixtime, year, desc, collect_list, collect_set, array_intersect, size, round, sum as spark_sum, when, coalesce, lit

In [ ]:
spark = SparkSession.builder.appName("Lab 3").getOrCreate()

In [ ]:
customer_list_path ="/content/drive/MyDrive/ds200/Lab3/data/Customer_List.csv"
order_items_path ="/content/drive/MyDrive/ds200/Lab3/data/Order_Items.csv"
order_reviews_path ="/content/drive/MyDrive/ds200/Lab3/data/Order_Reviews.csv"
orders_path ="/content/drive/MyDrive/ds200/Lab3/data/Orders.csv"
products_path ="/content/drive/MyDrive/ds200/Lab3/data/Products.csv"

#### 1.	Hãy đọc dữ liệu từ các file csv, sử dụng tự suy ra kiểu dữ liệu cho mỗi cột.

In [ ]:
customer_list_df = spark.read.csv(customer_list_path, header=True, inferSchema=True, sep=";")
order_items_df = spark.read.csv(order_items_path, header=True, inferSchema=True, sep=";")
order_reviews_df = spark.read.csv(order_reviews_path, header=True, inferSchema=True, sep=";")
orders_df = spark.read.csv(orders_path, header=True, inferSchema=True, sep=";")
products_df = spark.read.csv(products_path, header=True, inferSchema=True, sep=";")

In [ ]:
print("-----------------------------------------------------------------------")
print("customer_list_df")
customer_list_df.printSchema()
print("-----------------------------------------------------------------------")
print("order_items_df")
order_items_df.printSchema()
print("-----------------------------------------------------------------------")
print("order_reviews_df")
order_reviews_df.printSchema()
print("-----------------------------------------------------------------------")
print("orders_df")
orders_df.printSchema()
print("-----------------------------------------------------------------------")
print("products_df")
products_df.printSchema()
print("-----------------------------------------------------------------------")

-----------------------------------------------------------------------
customer_list_df
root
 |-- Customer_Trx_ID: string (nullable = true)
 |-- Subscriber_ID: string (nullable = true)
 |-- Subscribe_Date: date (nullable = true)
 |-- First_Order_Date: date (nullable = true)
 |-- Customer_Postal_Code: string (nullable = true)
 |-- Customer_City: string (nullable = true)
 |-- Customer_Country: string (nullable = true)
 |-- Customer_Country_Code: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)

-----------------------------------------------------------------------
order_items_df
root
 |-- Order_ID: string (nullable = true)
 |-- Order_Item_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Seller_ID: string (nullable = true)
 |-- Shipping_Limit_Date: timestamp (nullable = true)
 |-- Price: double (nullable = true)
 |-- Freight_Value: double (nullable = true)

-----------------------------------------------------

#### 2.	Thống kê tổng số đơn hàng, số lượng khách hàng và người bán.

In [ ]:
total_orders = orders_df.count()
total_customers = customer_list_df.distinct().count()
total_sellers = order_items_df.select("Seller_ID").distinct().count()

print("Tổng số đơn hàng:", total_orders)
print("Tổng số khách hàng:", total_customers)
print("Tổng số người bán:", total_sellers)

Tổng số đơn hàng: 99441
Tổng số khách hàng: 102727
Tổng số người bán: 3095


#### 3.	Phân tích số lượng đơn hàng theo quốc gia, sắp xếp theo thứ tự giảm dần.

In [ ]:
total_orders_groupby_country_df = orders_df.join(customer_list_df.select("Customer_Trx_ID", "Customer_Country"), on="Customer_Trx_ID", how="left")
total_orders_groupby_country = total_orders_groupby_country_df.groupBy("Customer_Country").agg(count("*").alias("Total_Orders")).orderBy(desc("Total_Orders"))
total_orders_groupby_country.show(n=50, truncate=False)

+----------------+------------+
|Customer_Country|Total_Orders|
+----------------+------------+
|Germany         |41754       |
|France          |12848       |
|Netherlands     |11629       |
|Belgium         |5464        |
|Austria         |5043        |
|Switzerland     |3640        |
|United Kingdom  |3382        |
|Poland          |2139        |
|Czechia         |2034        |
|Italy           |2025        |
|Spain           |1651        |
|Portugal        |1336        |
|Sweden          |975         |
|Denmark         |905         |
|Serbia          |746         |
|Norway          |716         |
|Slovakia        |534         |
|Slovenia        |495         |
|Turkey          |485         |
|Greece          |412         |
|Lithuania       |351         |
|Latvia          |280         |
|Croatia         |254         |
|Estonia         |148         |
|Finland         |81          |
|Luxembourg      |68          |
|Andorra         |46          |
+----------------+------------+



#### 4.	Phân tích số lượng đơn hàng nhóm theo năm, tháng đặt hàng (Hiển thị theo năm tăng dần, tháng giảm dần).

In [ ]:
orders_with_year_month_df = orders_df.withColumn("Year", year("Order_Purchase_Timestamp")).withColumn("Month", month("Order_Purchase_Timestamp")).select("Order_ID", "Year", "Month")
orders_grouped = orders_with_year_month_df.groupBy("Year", "Month").agg(count("Order_ID").alias("Total_Orders"))
orders_sorted = orders_grouped.orderBy("Year", desc("Month"))
orders_sorted.show(n=50, truncate=False)

+----+-----+------------+
|Year|Month|Total_Orders|
+----+-----+------------+
|2022|12   |1           |
|2022|10   |324         |
|2022|9    |4           |
|2023|12   |5673        |
|2023|11   |7544        |
|2023|10   |4631        |
|2023|9    |4285        |
|2023|8    |4331        |
|2023|7    |4026        |
|2023|6    |3245        |
|2023|5    |3700        |
|2023|4    |2404        |
|2023|3    |2682        |
|2023|2    |1780        |
|2023|1    |800         |
|2024|10   |4           |
|2024|9    |16          |
|2024|8    |6512        |
|2024|7    |6292        |
|2024|6    |6167        |
|2024|5    |6873        |
|2024|4    |6939        |
|2024|3    |7211        |
|2024|2    |6728        |
|2024|1    |7269        |
+----+-----+------------+



#### 5.	Thống kê điểm đánh giá trung bình, số lượng đánh giá theo từng mức (ví dụ: 1 đến 5).

In [ ]:
reviews_cleaned = order_reviews_df.withColumn("Review_Score_Int", col("Review_Score").cast("int"))
reviews_filtered = reviews_cleaned.filter((col("Review_Score_Int").isNotNull()) & (col("Review_Score_Int") >= 1) & (col("Review_Score_Int") <= 5))
overall_avg = reviews_filtered.agg(avg("Review_Score_Int")).first()[0]
review_stats = reviews_filtered.groupBy("Review_Score_Int").agg(count("*").alias("Total_Reviews")).orderBy("Review_Score_Int").collect()

print(f"- Average Review Score: {overall_avg}")
print("- Thống kê review_score:")
for row in review_stats:
    print(f"  Rating {row['Review_Score_Int']} : {row['Total_Reviews']}")

- Average Review Score: 4.0864214950162765
- Thống kê review_score:
  Rating 1 : 11424
  Rating 2 : 3151
  Rating 3 : 8179
  Rating 4 : 19141
  Rating 5 : 57328


#### 6.	Tính doanh thu (giá sản phẩm + phí vận chuyển) trong năm 2024 và nhóm theo danh mục sản phẩm.

In [ ]:
order_items_with_date = order_items_df.join(orders_df.select("Order_ID", "Order_Purchase_Timestamp"), on="Order_ID", how="inner")
order_items_2024 = order_items_with_date.filter(year("Order_Purchase_Timestamp") == 2024)
items_with_category = order_items_2024.join(products_df.select("Product_ID", "Product_Category_Name"), on="Product_ID", how="left")
items_with_revenue = items_with_category.withColumn("Revenue", col("Price") + col("Freight_Value"))
revenue_by_category = items_with_revenue.groupBy("Product_Category_Name").agg(spark_sum("Revenue").alias("Total_Revenue")).orderBy(col("Total_Revenue").desc())
revenue_by_category.show(n=100, truncate=False)

+---------------------------------------+------------------+
|Product_Category_Name                  |Total_Revenue     |
+---------------------------------------+------------------+
|Health_Beauty                          |885191.119999997  |
|Watches_Gifts                          |771986.750000001  |
|Bed_Bath_Table                         |650794.700000002  |
|Sports_Leisure                         |621999.3399999994 |
|Computers_Accessories                  |594771.0400000002 |
|Housewares                             |491576.9600000012 |
|Furniture_Decor                        |476466.1300000007 |
|Auto                                   |404210.5700000002 |
|Baby                                   |299052.5599999998 |
|Cool_Stuff                             |273910.0500000001 |
|Garden_Tools                           |259068.31999999983|
|Telephony                              |217452.1299999995 |
|Perfumery                              |204562.53999999992|
|Toys                   